In [1]:
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.models import Sequential

from keras.applications.densenet import DenseNet169
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import load_model
from keras.utils.vis_utils import plot_model

import pandas as pd
import numpy as np
from pathlib import Path

import matplotlib
matplotlib.use("Agg") # set the matplotlib backend so figures can be saved in the background
 
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix

from time import time
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
pathFileTrain = "CheXpert-v1.0-small/train.csv"
pathFileValid = "CheXpert-v1.0-small/valid.csv"

# import train data from train.csv into panda struct
dtrain = pd.read_csv(pathFileTrain)
# fill in blanks with 0s
dtrain = dtrain.fillna(0)

# import valid data from valid.csv into panda struct
dvalid = pd.read_csv(pathFileValid)
# fill in blanks with 0s
dvalid = dvalid.fillna(0)

# combine train and valid dataset
dtrain = dtrain.append(dvalid)

# Keep the lateral images

# drop the columns for sex, age, frontal/lateral, and AP/PA
dtrain = dtrain.drop(["Sex", "Age", "Frontal/Lateral", "AP/PA"], axis=1)

# convert uncertain to positive (-1 to 1)
dtrain = dtrain.replace(-1,1)

print(dtrain.shape)
dtrain.describe().transpose()

(223648, 15)


,count,mean,std,min,25%,50%,75%,max
No Finding,223648.0,0.100242,0.300324,0.0,0.0,0.0,0.0,1.0
Enlarged Cardiomediastinum,223648.0,0.104226,0.305555,0.0,0.0,0.0,0.0,1.0
Cardiomegaly,223648.0,0.157189,0.363980,0.0,0.0,0.0,0.0,1.0
Lung Opacity,223648.0,0.497679,0.499996,0.0,0.0,0.0,1.0,1.0
Lung Lesion,223648.0,0.047731,0.213198,0.0,0.0,0.0,0.0,1.0
Edema,223648.0,0.291865,0.454621,0.0,0.0,0.0,1.0,1.0
Consolidation,223648.0,0.190290,0.392531,0.0,0.0,0.0,0.0,1.0
Pneumonia,223648.0,0.110965,0.314089,0.0,0.0,0.0,0.0,1.0
Atelectasis,223648.0,0.300450,0.458455,0.0,0.0,0.0,1.0,1.0
Pneumothorax,223648.0,0.101056,0.301404,0.0,0.0,0.0,0.0,1.0


In [3]:
# Split data into train/dev/test 0.8/0.1/0.1

# Randomize the data
dtotal = dtrain.sample(frac=1)

# Split data into traing (80%), valid (10%) and test (10%)
dvalid_size = round(0.1*dtotal.shape[0])
dtest_size = dvalid_size
dtrain_size = dtotal.shape[0] - dvalid_size - dtest_size

x_train = dtotal[0:dtrain_size]
x_valid = dtotal[dtrain_size:dtrain_size+dvalid_size]
x_test = dtotal[dtrain_size+dvalid_size:dtotal.shape[0]+1]

print("The total number of data is:", dtotal.shape[0])

print("The size of training set is:", x_train.shape[0])
print("The size of the dev set is:", x_valid.shape[0])
print("The size of the test set is:", x_test.shape[0])

The total number of data is: 223648
The size of training set is: 178918
The size of the dev set is: 22365
The size of the test set is: 22365


In [4]:
# Data generation 
# https://keras.io/preprocessing/image/#imagedatagenerator-class
train_datagen = ImageDataGenerator(rescale = 1./255)
valid_datagen = ImageDataGenerator(rescale = 1./255.)
test_datagen = ImageDataGenerator(rescale = 1./255.)

target_size = (224, 224)

train_generator = train_datagen.flow_from_dataframe(
        dataframe = x_train,
        directory = None,
        x_col = "Path",
        y_col = list(x_train.columns[1:15]),
        class_mode = "other",
        target_size = target_size,
        batch_size = 32)

valid_generator = valid_datagen.flow_from_dataframe(
        dataframe = x_valid,
        directory = None,
        x_col = "Path",
        y_col = list(x_valid.columns[1:15]),
        class_mode = "other",
        target_size = target_size,
        batch_size = 32)

test_generator = test_datagen.flow_from_dataframe(
        dataframe = x_test,
        directory = None,
        x_col = "Path",
        y_col = list(x_test.columns[1:15]),
        class_mode = "other",
        target_size = target_size,
        batch_size = 1)

Found 178918 validated image filenames.
Found 22365 validated image filenames.
Found 22365 validated image filenames.


In [5]:
# Creating the model architecture
# Use the DenseNet169 model with pretrained weights from imagenet. Exclude the top layers.
model = DenseNet169(include_top = False, weights='imagenet')
layer1 = model.output
# adding a 2D pooling layer
layer1 = GlobalAveragePooling2D()(layer1)
# adding a fully connected layer
layer1 = Dense(1024, activation='relu')(layer1)
# adding a fully connected layer
layer1 = Dense(512, activation='relu')(layer1)
# adding a fully connected layer
layer1 = Dense(256, activation='relu')(layer1)
# adding a softmax layer
predictions = Dense(14, activation='sigmoid')(layer1)

Model_Dense169 = Model(inputs=model.input, outputs=predictions)

In [6]:
# Use Adam optimization
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

Model_Dense169.compile(optimizer= adam, loss='binary_crossentropy', metrics=['accuracy'])

print(Model_Dense169.summary())


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
______________

In [ ]:
# Fitting data into the model architecture

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
FullModel = Model_Dense169.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=3)

# Save the model!
Model_Dense169.save("Model_DenseNet169_FCs_frac=1_epoch=3_lateral_frontal.h5")

In [ ]:
# Plotting the loss curve
plt.plot(FullModel.history['loss'])
plt.plot(FullModel.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.savefig("losscurve_attempt7")

In [7]:
model_Dense = load_model("Model_DenseNet169_FCs_frac=1_epoch=3_lateral_frontal.h5")
num_epochs = 3
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [8]:
# Assess performance
test_generator.reset()
pred = model_Dense.predict_generator(test_generator, steps=STEP_SIZE_TEST)
pred_bool = (pred >= 0.5)

y_pred = np.array(pred_bool,dtype =int)

dtest = x_test.to_numpy()
y_true = np.array(dtest[:,1:15],dtype=int)

print(classification_report(y_true, y_pred,target_names=list(x_train.columns[1:15])))

                            precision    recall  f1-score   support

                No Finding       0.11      0.03      0.05      2266
Enlarged Cardiomediastinum       0.17      0.00      0.01      2379
              Cardiomegaly       0.17      0.08      0.11      3631
              Lung Opacity       0.49      0.47      0.48     11094
               Lung Lesion       0.06      0.00      0.01      1070
                     Edema       0.29      0.25      0.27      6503
             Consolidation       0.19      0.01      0.03      4219
                 Pneumonia       0.13      0.01      0.02      2442
               Atelectasis       0.30      0.14      0.19      6704
              Pneumothorax       0.10      0.07      0.08      2220
          Pleural Effusion       0.43      0.46      0.44      9691
             Pleural Other       0.06      0.00      0.00       613
                  Fracture       0.05      0.01      0.01       940
           Support Devices       0.53      0.55

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [9]:
score, accuracy = model_Dense.evaluate_generator(test_generator, steps=STEP_SIZE_TEST)
print('Test score:', score)
print('Test accuracy:', accuracy)

Test score: 0.32783227786605
Test accuracy: 0.8597585444905576
